In [ ]:
import pandas as pd
import datetime
import itertools

users = pd.DataFrame(columns=['UserId', 'Name', 'Email', 'Password', 'Phone', 'MailingAddress', 'BillingAddress', 'PreferredDinerNumber', 'EarnedPoints', 'PreferredPaymentMethod'])
reservations = pd.DataFrame(columns=['UserId', 'Name', 'Phone', 'Email', 'DateTime', 'Guests', 'TableIds', 'PaymentMethod', 'MailingAddress', 'BillingAddress'])
tables = pd.DataFrame(columns=['TableId', 'Capacity'])

def validate_phone(phone):
    if len(phone) != 10 or not phone.isdigit():
        return False
    return True


def validate_email(email):
    if "@" not in email or ".com" not in email:
        return False
    return True

def register_user():
    user_id = len(users) + 1
    name = input("Enter your name: ")
    phone = input("Enter your phone number (10 digits): ")
    while not validate_phone(phone):
        print("Invalid phone number. Please enter a 10 digit number.")
        phone = input("Enter your phone number (10 digits): ")

    email = input("Enter your email address: ")
    while not validate_email(email):
        print("Invalid email address. Please ensure the email contains '@' and ends with '.com'.")
        email = input("Enter your email address: ")
    password = input("Enter your password: ")
    mailing_address = input("Enter your mailing address: ")
    billing_address = input("Enter your billing address (press enter if same as mailing address): ")
    if not billing_address:
        billing_address = mailing_address
    preferred_diner_number = len(users) + 1
    earned_points = 0
    preferred_payment_method = input("Enter your preferred payment method (cash, credit, check): ")
    
    # Validate email format
    if "@" not in email or ".com" not in email:
        print("Invalid email address format. Email address should contain '@' symbol and end with '.com'.")
        return
    
    new_user = {'UserId': user_id, 'Name': name, 'Email': email, 'Password': password, 'Phone': phone, 'MailingAddress': mailing_address, 'BillingAddress': billing_address, 'PreferredDinerNumber': preferred_diner_number, 'EarnedPoints': earned_points, 'PreferredPaymentMethod': preferred_payment_method}
    return new_user


def login():
    email = input("Enter your email address: ")
    password = input("Enter your password: ")
    user = users.loc[(users['Email'] == email) & (users['Password'] == password)]
    if not user.empty:
        return user.iloc[0]
    else:
        print("User not found or incorrect password.")
        return None

def is_high_traffic(date_time):
    # Customize the high-traffic hours and dates according to your needs
    high_traffic_hours = range(18, 22)
    high_traffic_dates = [datetime.date(year=date_time.year, month=7, day=4)]

    if date_time.date() in high_traffic_dates or (date_time.weekday() < 5 and date_time.hour in high_traffic_hours):
        return True
    return False

def get_payment_method(user):
    if user is not None:
        return user['PreferredPaymentMethod']
    return input("Enter your payment method (cash, credit, check): ")

def create_tables(table_reservations):
    table_list = []
    for i, reservation in enumerate(table_reservations, start=1):
        table_id = str(i)
        capacity = reservation['Capacity']
        table_list.append({'TableId': table_id, 'Capacity': capacity})
    return pd.DataFrame(table_list)

def reserve_tables(guests):
    # Check if all tables are available
    if len(tables) >= guests:
        table_ids = tables.head(guests)['TableId'].tolist()
        return table_ids

    # Order tables based on capacity
    sorted_tables = tables.sort_values(by='Capacity', ascending=False).copy()

    reserved_table_ids = []
    remaining_guests = guests

    while remaining_guests > 0:
        found_table = False
        for _, table in sorted_tables.iterrows():
            if table['Capacity'] <= remaining_guests and table['TableId'] not in reserved_table_ids:
                reserved_table_ids.append(table['TableId'])
                remaining_guests -= table['Capacity']
                found_table = True
                if remaining_guests == 0:
                    break

        if not found_table:
            combination = combine_tables(sorted_tables, reserved_table_ids, remaining_guests)
            if combination:
                reserved_table_ids.extend(combination)
                remaining_guests = 0
            else:
                print("We're unable to accommodate your party.")
                return None

    # Remove reserved tables from the list of available tables
    tables.drop(tables[tables['TableId'].isin(reserved_table_ids)].index, inplace=True)
    tables.reset_index(drop=True, inplace=True)

    return reserved_table_ids

def combine_tables(sorted_tables, reserved_table_ids, remaining_guests):
    table_ids = []
    table_capacities = []

    for _, table in sorted_tables.iterrows():
        if table['TableId'] not in reserved_table_ids:
            table_ids.append(table['TableId'])
            table_capacities.append(table['Capacity'])

    combinations = []
    for i in range(1, len(table_ids) + 1):
        for combination in itertools.combinations(range(len(table_ids)), i):
            capacity_sum = sum(table_capacities[j] for j in combination)
            if capacity_sum >= remaining_guests:
                combinations.append([table_ids[j] for j in combination])

    # Sort combinations by the number of tables used
    combinations.sort(key=len)

    # Select the combination with the minimum number of tables
    if combinations:
        return combinations[0]

    return None

def create_reservation(user, is_guest):
    if is_guest:
        name = input("Enter your name: ")
        phone = input("Enter your phone number: ")
        while not validate_phone(phone):
            print("Invalid phone number. Please enter a 10 digit number.")
            phone = input("Enter your phone number: ")

        email = input("Enter your email address: ")
        while not validate_email(email):
            print("Invalid email address. Please ensure the email contains '@' and ends with '.com'.")
            email = input("Enter your email address: ")
        mailing_address = None
        billing_address = None

        # Validate the number of guests
        while True:
            guests = input("Enter the number of guests (1-16): ")
            if guests.isdigit():
                guests = int(guests)
                if guests < 1 or guests > 16:
                    print("Invalid number of guests. Please enter a value between 1 and 16.")
                else:
                    break
            else:
                print("Invalid input. Please enter a numerical value for the number of guests.")
    else:
        name = user['Name']
        phone = user['Phone']
        email = user['Email']
        mailing_address = user['MailingAddress']
        billing_address = user['BillingAddress']

        # Validate email format
        if "@" not in email or ".com" not in email:
            print("Invalid email address format. Email address should contain '@' symbol and end with '.com'.")
            return

        # Validate phone number format
        if not phone.isdigit() or len(phone) != 10:
            print("Invalid phone number. Phone number should be a 10-digit numerical value.")
            return

        # Validate the number of guests
        while True:
            guests = input("Enter the number of guests (1-16): ")
            if guests.isdigit():
                guests = int(guests)
                if guests < 1 or guests > 16:
                    print("Invalid number of guests. Please enter a value between 1 and 16.")
                else:
                    break
            else:
                print("Invalid input. Please enter a numerical value for the number of guests.")

    date = input("Enter the date (YYYY-MM-DD): ")
    time = input("Enter the time (HH:MM): ")

    date_time = datetime.datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")

    user_id = None if is_guest else user['UserId']

    if date_time < datetime.datetime.now():
        print("Invalid date and time. Please enter a future date and time.")
        return

    if is_high_traffic(date_time):
        print("High-traffic hours detected. A valid payment method is required.")
        payment_method = get_payment_method(user)
    else:
        payment_method = None

    reserved_table_ids = reserve_tables(guests)

    if reserved_table_ids:
        reservation = {'UserId': user_id, 'Name': name, 'Phone': phone, 'Email': email, 'DateTime': date_time,
                       'Guests': guests, 'TableIds': reserved_table_ids, 'PaymentMethod': payment_method,
                       'MailingAddress': mailing_address, 'BillingAddress': billing_address}
        reservations.loc[len(reservations)] = reservation
        reservation_id = len(reservations) - 1  # Get the index of the newly added reservation
        if is_guest:
            print("Consider registering to enjoy additional benefits.")
        print(f"Reservation successful. Your reservation ID is: {reservation_id}")
    else:
        print("No tables available for your reservation.")


def display_reservation_details(reservation):
    if not reservation.empty:
        print("\nReservation Details:")
        print(f"Name: {reservation['Name']}")
        print(f"Phone: {reservation['Phone']}")
        print(f"Email: {reservation['Email']}")
        print(f"Date and Time: {reservation['DateTime'].strftime('%Y-%m-%d %H:%M')}")
        print(f"Number of Guests: {reservation['Guests']}")
        print(f"Table(s): {', '.join(str(table_id) for table_id in reservation['TableIds'])}")
        if reservation['MailingAddress']:
            print(f"Mailing Address: {reservation['MailingAddress']}")
        if reservation['BillingAddress']:
            print(f"Billing Address: {reservation['BillingAddress']}")
    else:
        print("Reservation not found.")

def main():
    global users, reservations, tables

    while True:
        print("\nOptions:")
        print("1. Register user")
        print("2. Login and reserve a table")
        print("3. Guest reservations")
        print("4. Show reservation details")
        print("5. Show table capacities")
        print("6. Exit")

        option = int(input("Select an option (1-6): "))

        if option == 1:
            users.loc[len(users)] = register_user()
            print("User registration successful.")

        elif option == 2:
            user = login()
            if user is not None:
                create_reservation(user, False)

        elif option == 3:
            create_reservation(None, True)

        elif option == 4:
            reservation_id = int(input("Enter the reservation ID: "))
            reservation = reservations.loc[reservation_id]
            if not reservation.empty:
                display_reservation_details(reservation)
            else:
                print("Reservation not found.")

        elif option == 5:
            print("Table capacities:")
            print(tables)

        elif option == 6:
            print("Exiting the program.")
            break

        else:
            print("Invalid option. Please select a valid option (1-6).")

if __name__ == "__main__":
    # Create tables from table reservations
    table_reservations = [
        {'TableId': '1', 'Capacity': 2},
        {'TableId': '2', 'Capacity': 2},
        {'TableId': '3', 'Capacity': 4},
        {'TableId': '4', 'Capacity': 4},
        {'TableId': '5', 'Capacity': 6},
        {'TableId': '6', 'Capacity': 6},
        {'TableId': '7', 'Capacity': 8},
        {'TableId': '8', 'Capacity': 8},
        {'TableId': '9', 'Capacity': 10},
        {'TableId': '10', 'Capacity': 10}
    ]
    tables = create_tables(table_reservations)

    main()
   


Options:
1. Register user
2. Login and reserve a table
3. Guest reservations
4. Show reservation details
5. Show table capacities
6. Exit
Select an option (1-6): 1
Enter your name: nahid ferdous
Enter your phone number (10 digits): 123123
Invalid phone number. Please enter a 10 digit number.
Enter your phone number (10 digits): 3462778750
Enter your email address: nasdas
Invalid email address. Please ensure the email contains '@' and ends with '.com'.
Enter your email address: nahid@gmail.com
Enter your password: 12345
Enter your mailing address: abcd
Enter your billing address (press enter if same as mailing address): asdasd
Enter your preferred payment method (cash, credit, check): cash
User registration successful.

Options:
1. Register user
2. Login and reserve a table
3. Guest reservations
4. Show reservation details
5. Show table capacities
6. Exit
Select an option (1-6): 2
Enter your email address: nahid@gmail.com
Enter your password: 12345
Enter the number of guests (1-16): 14